# [Issue# 1897 Additional Visuals for PUC Analysis](https://github.com/cal-itp/data-analyses/issues/1897)

Received list of transit operators cohorts that may be exempt from efficiency reporting, per PUC 99314.11, .6 and .7. 
- create visuals based on grouping set by list
- recreate visuals based on previous notebook work

## [99314.6](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=99314.6.&lawCode=PUC)
>`funds shall be allocated for operating or capital purpose` pursuant to Sections 99313 and 99314 to an operator `if the operator meets either of the following efficiency standards`:
>- (A) `The operator shall receive its entire allocation`, and any or all of this allocation may be used for operating purposes, if the operator’s `total operating cost per revenue vehicle hour` in the latest year for which audited data are available `does not exceed the sum of the preceding year’s total operating cost per revenue vehicle hour and an amount equal to the product of the percentage change in the Consumer Price Index for the same period multiplied by the preceding year’s total operating cost per revenue vehicle hour.`
>- (B) The operator shall receive its entire allocation, and any or all of this allocation may be used for operating purposes, `if the operator’s average total operating cost per revenue vehicle hour` in the latest three years for which audited data are available `does not exceed the sum of the average of the total operating cost per revenue vehicle hour in the three years preceding the latest year for which audited data are available and an amount equal to the product of the average percentage change in the Consumer Price Index for the same period multiplied by the average total operating cost per revenue vehicle hour in the same three years`.
## [99314.7 (mainly MTC specific)](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PUC&sectionNum=99314.7.)
>the `Metropolitan Transportation Commission` shall apply the following eligibility standards to the operators within the region subject to its jurisdiction:

# [99314.11](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=99314.11.&nodeTreePath=17.11.2.8&lawCode=PUC)
>`Sections 99314.6 and 99314.7 do not apply to an operator for a fiscal year in which the operator expended from local funding an amount for transit operations not less than the amount the operator expended from local funding for transit operations during the 2018–19 fiscal year.` As used in this subdivision, “local funding” means any nonstate grant funds or other revenues generated by, earned by, or distributed to, an operator.

Meaning, if a transit operator spent local funds >= the local funds spent during FY 2018-2019, they are exempt from meeting efficiency standards(?)

## Data Exploration

### Categorical variaables
- Underlying metric
  - Farebox Recovery Ratio
  - Local funding expended
- area type
  - urban
  - rural
- cohorts
  - A
  - B
  - C
- NTD metric
  - UPT
  - PMT
  - VRH
- year
  - 2019
  - 2020
  - 2021
  - 2022
  - 2023
  - 2024

## analyses should be split by underlying metric
resulting groups are:
1. Farebox Recovery ratio
    - urban
        - cohorts
        - ntd metric
        - year
    - rural
        - cohorts
        - ntd metrics
        - year
2. Local funding expended
    - urban
        - cohorts
        - ntd metric
        - year
    - rural
        - cohorts
        - ntd metrics
        - year

## 



In [1]:
import pandas as pd
import altair as alt
from functools import cache
from calitp_data_analysis.gcs_pandas import GCSPandas
from calitp_data_analysis.sql import get_engine, to_snakecase, query_sql

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

@cache
def gcs_pandas():
    return GCSPandas()

gcs_path = "gs://calitp-analytics-data/data-analyses/ntd/"

# Read in cohort list data

In [2]:
# cohort_data = gcs_pandas().read_csv(f"{gcs_path}fbr_local_funding_by_cohorts_2019-2024_compiled.csv")

# cohort_data.columns = cohort_data.columns.str.lower()
# cohort_data["ntd_id"] = cohort_data["ntd_id"].astype("str")

# display(
#     cohort_data.info(),
#     cohort_data.head(),
#     cohort_data.value_counts(
#     subset=["urban_rural","metric","cohort","year"]
#     )
# )

# Read in yes/no list data

In [3]:
yes_no_data = gcs_pandas().read_csv(f"{gcs_path}cs_sco_yes_no_fbr_funding_2019-2024.csv")

yes_no_data.columns = yes_no_data.columns.str.lower()
yes_no_data[["year","ntd_id"]] = yes_no_data[["year","ntd_id"]].astype("str")

display(
    yes_no_data.info(),
    yes_no_data.head(3)
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1797 entries, 0 to 1796
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   year              1797 non-null   object
 1   ntd_id            1797 non-null   object
 2   ntd_entity_name   1797 non-null   object
 3   area_type         1797 non-null   object
 4   metric            1797 non-null   object
 5   quartile          1797 non-null   object
 6   metric_short      1797 non-null   object
 7   metric_value      1797 non-null   object
 8   requirement       1797 non-null   object
 9   requirement_flag  1797 non-null   object
dtypes: object(10)
memory usage: 140.5+ KB


None

,year,ntd_id,ntd_entity_name,area_type,metric,quartile,metric_short,metric_value,requirement,requirement_flag
0,2019,90003,San Francisco Bay Area Rapid Transit District,Urban,Farebox Recovery Ratio,Top 25%,FBR,63.14,Met FBR Min,YES
1,2019,90003,San Francisco Bay Area Rapid Transit District,Urban,Local Funding % Change vs 2019,Middle 50%,Pct_Change_vs_2019,0.014763266,Maintained_or_Increased_vs_2019,Yes
2,2019,90004,Golden Empire Transit District,Urban,Farebox Recovery Ratio,Middle 50%,FBR,20.67,Met FBR Min,YES


# Read in analysis data from prev notebook

In [4]:

# ntd_name = "puc_analysis_data.parquet"
# ntd_analysis_data = gcs_pandas().read_parquet(f"{gcs_path}{ntd_name}")

# display(
#     ntd_analysis_data.info(),
#     ntd_analysis_data["year"].unique()
# )

# May need to requery this data to include 2024
is 2024 NTD data in the warehouse now? copy pasted from initial puc analysis notebook.

In [5]:
metric_list = [
    "pmt",
    "upt",
    "vrh",
    # "opexp_total" # not needed for this project
]

# empty list for appending DFs
df_list = []

# loop to query pmt, upt and vrh from 2018 to 2024
for metric in metric_list:
        query = f"""
        SELECT
          ntd_id,
          source_agency,
          agency_status,
          primary_uza_name,
          uza_population,
          uza_area_sq_miles,
          year,
          mode,
          type_of_service,
          reporter_type,
          SUM({metric}) AS total_{metric},
        FROM
          `cal-itp-data-infra.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_{metric}`
        WHERE
          source_state = "CA"
          AND year BETWEEN 2018 AND 2024
        GROUP BY
          ntd_id,
          source_agency,
          agency_status,
          primary_uza_name,
          uza_population,
          uza_area_sq_miles,
          year,
          mode,
          type_of_service,
          reporter_type
        """
        # create df
        metric = query_sql(query, as_df=True)

        # append df to list
        df_list.append(metric)

# unpack list into separate DFs
ntd_pmt, ntd_upt, ntd_vrh = df_list

display( 
    ntd_upt.head(3)
)

,ntd_id,source_agency,agency_status,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,type_of_service,reporter_type,total_upt
0,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.80,2019,MG,PT,Full Reporter,"886,515.00"
1,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.80,2018,MB,PT,Full Reporter,NaN
2,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.80,2020,MB,PT,Full Reporter,NaN


## merge all the metrics together

In [6]:
merge_on_col = [
    "ntd_id",
    "year",
    "source_agency",
    "agency_status",
    "primary_uza_name",
    "uza_population",
    "uza_area_sq_miles",
    "mode",
    "type_of_service",
    "reporter_type",
]

merge_1 = ntd_vrh.merge(ntd_upt, on=merge_on_col, how="inner")
# merge_2 = merge_1.merge(ntd_vrh, on=merge_on_col, how = "inner")

ntd_metrics_merge = merge_1.merge(ntd_pmt, on=merge_on_col, how="inner")

ntd_metrics_merge.head(3)

,ntd_id,source_agency,agency_status,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,type_of_service,reporter_type,total_vrh,total_upt,total_pmt
0,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.80,2019,MG,PT,Full Reporter,"19,815.00","886,515.00","2,819,118.00"
1,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.80,2019,MB,PT,Full Reporter,NaN,NaN,NaN
2,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.80,2021,MG,PT,Full Reporter,"17,819.00","112,981.00","359,280.00"


## get districts for ntd ID
- Do i still need district data for this specific analysis?

In [7]:
for metric in metric_list:
        query = f"""
        SELECT
          `mart_transit_database.dim_organizations`.`key` AS `key`,
          `mart_transit_database.dim_organizations`.`source_record_id` AS `source_record_id`,
          `mart_transit_database.dim_organizations`.`name` AS `name`,
          `mart_transit_database.dim_organizations`.`ntd_id_2022` AS `ntd_id_2022`,
          `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`county_geography_name` AS `county`,
          `Dim_County_Geography___County_Geography_Key`.`caltrans_district` AS `caltrans_district`
        FROM
          `mart_transit_database.dim_organizations`

        LEFT JOIN `mart_transit_database.bridge_organizations_x_headquarters_county_geography` AS `Bridge_Organizations_X_Headquarters_County_Geography___Key` ON `mart_transit_database.dim_organizations`.`key` = `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`organization_key`
          LEFT JOIN `mart_transit_database.dim_county_geography` AS `Dim_County_Geography___County_Geography_Key` ON `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`county_geography_key` = `Dim_County_Geography___County_Geography_Key`.`key`
        WHERE
          (
            `mart_transit_database.dim_organizations`.`_is_current` = TRUE
          )

           AND (
            `mart_transit_database.dim_organizations`.`ntd_id_2022` IS NOT NULL
          )
          AND (
            (
              `mart_transit_database.dim_organizations`.`ntd_id_2022` <> ''
            )

            OR (
              `mart_transit_database.dim_organizations`.`ntd_id_2022` IS NULL
            )
          )
          AND (
            `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`_is_current` = TRUE
          )
          AND (
            `Dim_County_Geography___County_Geography_Key`.`_is_current` = TRUE
          )
        """
        # create df
        ntd_id_x_district = query_sql(query, as_df=True)
        
ntd_id_x_district["caltrans_district"] = ntd_id_x_district["caltrans_district"].astype("str")

ntd_id_x_district.head()

,key,source_record_id,name,ntd_id_2022,county,caltrans_district
0,d84a961daa618c733f9d9c3bd49c322f,recJtH0Ae8YNo01aj,Access Services,90157,Los Angeles,7
1,9b5971d16d58e4fcafa694ee7fa33b12,rec79AM4tMwdokWhE,Alpine County,91116,Alpine,10
2,e5de5083d68e8c2463a784ceb13e91f2,recUTSH4TT1wB3RSC,Attentive Transportation LLC,90314,Sacramento,3
3,957618c89db2f5e992caa5ca2e6086ab,rec7F6JKLMVrRhQJU,Bishop Paiute Tribe,99268,Inyo,9
4,a024fabd0002f9c9bd636042de30715d,recE6qJFuoREa9EHg,Calaveras County,91063,Calaveras,10


## merge the ntd metrics with Caltrans Districts

In [8]:
ntd_metrics_merge = ntd_metrics_merge.merge(
    ntd_id_x_district[["ntd_id_2022","county","caltrans_district"]],
    left_on = "ntd_id",
    right_on = "ntd_id_2022",
    how="inner",
    indicator=True
)
ntd_metrics_merge["year"] = ntd_metrics_merge["year"].astype("str")
ntd_metrics_merge.head()

,ntd_id,source_agency,agency_status,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,type_of_service,reporter_type,total_vrh,total_upt,total_pmt,ntd_id_2022,county,caltrans_district,_merge
0,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.80,2019,MG,PT,Full Reporter,"19,815.00","886,515.00","2,819,118.00",90003,San Francisco,4,both
1,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.80,2019,MB,PT,Full Reporter,NaN,NaN,NaN,90003,San Francisco,4,both
2,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.80,2021,MG,PT,Full Reporter,"17,819.00","112,981.00","359,280.00",90003,San Francisco,4,both
3,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.80,2021,DR,PT,Full Reporter,NaN,NaN,NaN,90003,San Francisco,4,both
4,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.80,2019,YR,DO,Full Reporter,"41,924.00","2,225,056.00","15,283,299.00",90003,San Francisco,4,both


# merge ntd metrics with ~~cohort data~~ yes/no data
- merge on ntd_id
- are there any unmerged rows?

In [9]:
ntd_metrics_merge.dtypes

ntd_id                 object
source_agency          object
agency_status          object
primary_uza_name       object
uza_population          int64
uza_area_sq_miles     float64
year                   object
mode                   object
type_of_service        object
reporter_type          object
total_vrh             float64
total_upt             float64
total_pmt             float64
ntd_id_2022            object
county                 object
caltrans_district      object
_merge               category
dtype: object

In [10]:
ntd_yes_no_merge = ntd_metrics_merge.drop(columns="_merge").merge(
    yes_no_data,
    left_on = ["ntd_id","year"],
    right_on = ["ntd_id","year"],
    indicator= True,
)

# any unmerged rows? NONE
ntd_yes_no_merge["_merge"].value_counts()

_merge
both          4965
left_only        0
right_only       0
Name: count, dtype: int64

In [17]:
# Sanity check
# pick up a couple of NTD ID, see if the merge data tracks with the cohort data
sample_ids = ntd_yes_no_merge["ntd_id"].sample(3).to_list()
keep_cols=[
    "ntd_id",
    "source_agency",
    "mode",
    "type_of_service",
    "total_vrh",
    "total_pmt",
    "total_upt",
    "area_type",
    "metric",
    "year",
    "requirement",
    "requirement_flag"
]

for sample_id in sample_ids:
    display(
        f"Sameple NTD ID: {sample_id}",
        "cohort data",
        yes_no_data[
            (yes_no_data["ntd_id"]== sample_id)
            & (yes_no_data["year"].isin(["2023","2024"]))
            ].sort_values(by=["area_type","metric","year"]).head(5),
        "merge table",
        ntd_yes_no_merge[
            (ntd_yes_no_merge["ntd_id"]== sample_id)
            & (ntd_yes_no_merge["year"].isin(["2023","2024"]))
            ][keep_cols].sort_values(by=["area_type","metric","year"]),
        
    )

# cohort data matches, 
# looks a little weird since the ntd metrics is per mode and TOS. the cohort data becomes categorical. GTG

'Sameple NTD ID: 90165'

'cohort data'

,year,ntd_id,ntd_entity_name,area_type,metric,quartile,metric_short,metric_value,requirement,requirement_flag
1321,2023,90165,City of Thousand Oaks,Urban,Farebox Recovery Ratio,Bottom 25%,FBR,4.3,Met FBR Min,NO
1620,2024,90165,City of Thousand Oaks,Urban,Farebox Recovery Ratio,Bottom 25%,FBR,5.31,Met FBR Min,NO
1322,2023,90165,City of Thousand Oaks,Urban,Local Funding % Change vs 2019,Middle 50%,Pct_Change_vs_2019,-0.379389092,Maintained_or_Increased_vs_2019,No
1621,2024,90165,City of Thousand Oaks,Urban,Local Funding % Change vs 2019,Top 25%,Pct_Change_vs_2019,0.041016526,Maintained_or_Increased_vs_2019,Yes


'merge table'

,ntd_id,source_agency,mode,type_of_service,total_vrh,total_pmt,total_upt,area_type,metric,year,requirement,requirement_flag
3786,90165,City of Thousand Oaks (TOT) - Public Works,DR,PT,"14,531.00",NaN,"39,523.00",Urban,Farebox Recovery Ratio,2023,Met FBR Min,NO
3788,90165,City of Thousand Oaks (TOT) - Public Works,MB,PT,"21,823.00",NaN,"152,351.00",Urban,Farebox Recovery Ratio,2023,Met FBR Min,NO
3790,90165,City of Thousand Oaks (TOT) - Public Works,DR,PT,"15,118.00",NaN,"45,316.00",Urban,Farebox Recovery Ratio,2024,Met FBR Min,NO
3792,90165,City of Thousand Oaks (TOT) - Public Works,MB,PT,"22,419.00",NaN,"154,287.00",Urban,Farebox Recovery Ratio,2024,Met FBR Min,NO
3787,90165,City of Thousand Oaks (TOT) - Public Works,DR,PT,"14,531.00",NaN,"39,523.00",Urban,Local Funding % Change vs 2019,2023,Maintained_or_Increased_vs_2019,No
3789,90165,City of Thousand Oaks (TOT) - Public Works,MB,PT,"21,823.00",NaN,"152,351.00",Urban,Local Funding % Change vs 2019,2023,Maintained_or_Increased_vs_2019,No
3791,90165,City of Thousand Oaks (TOT) - Public Works,DR,PT,"15,118.00",NaN,"45,316.00",Urban,Local Funding % Change vs 2019,2024,Maintained_or_Increased_vs_2019,Yes
3793,90165,City of Thousand Oaks (TOT) - Public Works,MB,PT,"22,419.00",NaN,"154,287.00",Urban,Local Funding % Change vs 2019,2024,Maintained_or_Increased_vs_2019,Yes


...

'Sameple NTD ID: 90031'

'cohort data'

,year,ntd_id,ntd_entity_name,area_type,metric,quartile,metric_short,metric_value,requirement,requirement_flag
1246,2023,90031,Riverside Transit Agency,Urban,Farebox Recovery Ratio,Top 25%,FBR,30.42,Met FBR Min,YES
1545,2024,90031,Riverside Transit Agency,Urban,Farebox Recovery Ratio,Top 25%,FBR,24.98,Met FBR Min,YES
1247,2023,90031,Riverside Transit Agency,Urban,Local Funding % Change vs 2019,Middle 50%,Pct_Change_vs_2019,-0.425983264,Maintained_or_Increased_vs_2019,No
1546,2024,90031,Riverside Transit Agency,Urban,Local Funding % Change vs 2019,Middle 50%,Pct_Change_vs_2019,-0.313105383,Maintained_or_Increased_vs_2019,No


'merge table'

,ntd_id,source_agency,mode,type_of_service,total_vrh,total_pmt,total_upt,area_type,metric,year,requirement,requirement_flag
3115,90031,Riverside Transit Agency (RTA),MB,PT,"142,582.00","7,996,677.00","775,941.00",Urban,Farebox Recovery Ratio,2023,Met FBR Min,YES
3125,90031,Riverside Transit Agency (RTA),CB,PT,NaN,NaN,NaN,Urban,Farebox Recovery Ratio,2023,Met FBR Min,YES
3127,90031,Riverside Transit Agency (RTA),CB,DO,"20,329.00","5,401,966.00","158,918.00",Urban,Farebox Recovery Ratio,2023,Met FBR Min,YES
3135,90031,Riverside Transit Agency (RTA),MB,DO,"331,580.00","26,180,098.00","4,087,331.00",Urban,Farebox Recovery Ratio,2023,Met FBR Min,YES
3141,90031,Riverside Transit Agency (RTA),DR,PT,"124,749.00","2,176,705.00","224,267.00",Urban,Farebox Recovery Ratio,2023,Met FBR Min,YES
3145,90031,Riverside Transit Agency (RTA),DR,TX,NaN,NaN,NaN,Urban,Farebox Recovery Ratio,2023,Met FBR Min,YES
3113,90031,Riverside Transit Agency (RTA),CB,PT,NaN,NaN,NaN,Urban,Farebox Recovery Ratio,2024,Met FBR Min,YES
3117,90031,Riverside Transit Agency (RTA),CB,DO,"20,827.00","6,400,193.00","185,874.00",Urban,Farebox Recovery Ratio,2024,Met FBR Min,YES
3123,90031,Riverside Transit Agency (RTA),MB,PT,"131,874.00","8,083,749.00","810,022.00",Urban,Farebox Recovery Ratio,2024,Met FBR Min,YES
3133,90031,Riverside Transit Agency (RTA),DR,TX,NaN,NaN,NaN,Urban,Farebox Recovery Ratio,2024,Met FBR Min,YES


# Save merged cohort data

In [ ]:
# cort_merge_filname = "ntd_cohort_data_2026-01-26.parquet"
# gcs_pandas().data_frame_to_parquet(ntd_cohort_merge,f"{gcs_path}{cort_merge_filname}")

# Save merged  yes/no data

In [20]:
yes_no_merge_filname = "ntd_yes/no_data_2026-01-28.parquet"
# gcs_pandas().data_frame_to_parquet(ntd_yes_no_merge,f"{gcs_path}{yes_no_merge_filname}")

# Read in merged ~~cohort~~ yes/no data from GCS

In [23]:
# ntd_cohort_merge = gcs_pandas().read_parquet(f"{gcs_path}{cort_merge_filname}")

ntd_yes_no_merge = gcs_pandas().read_parquet(f"{gcs_path}{yes_no_merge_filname}")

# separate list by both metrics (farebox and funding change)

In [ ]:
cohort_merge_farebox = ntd_cohort_merge[ntd_cohort_merge["metric"]=="Farebox Recovery Ratio"]
cohort_merge_funding = ntd_cohort_merge[ntd_cohort_merge["metric"]=="Local Funding % Change vs 2019"]

In [ ]:
display(
    cohort_merge_farebox.shape,
    cohort_merge_funding.shape,
    cohort_merge_farebox["metric"].unique(),
    cohort_merge_funding["metric"].unique(),
    cohort_merge_farebox.columns
)

# Group aggregation

## melt big DF 
- so all columns are under 1 column.

In [ ]:
group_list_melt = [
    "source_agency",
    "year",
    "ntd_id",
    "caltrans_district",
    "mode",
    "type_of_service",
    "urban_rural",
    "cohort",
    "metric"
]

value_cols = ["total_upt", "total_vrh", "total_pmt"]

melt_farebox = pd.melt(
    cohort_merge_farebox,
    id_vars=group_list_melt,
    value_vars=value_cols,
    var_name="ntd_metric",
    value_name="ntd_metric_value",
    ignore_index=True,
)

melt_funding = pd.melt(
    cohort_merge_funding,
    id_vars=group_list_melt,
    value_vars=value_cols,
    var_name="ntd_metric",
    value_name="ntd_metric_value",
    ignore_index=True,
)

In [ ]:
display(
    melt_farebox.shape,
    melt_funding.shape
)

In [ ]:
sample_ids = ntd_cohort_merge["ntd_id"].sample(3).to_list()
melt_farebox[melt_farebox["ntd_id"].isin([sample_ids[1]])].sort_values(by=["year","mode","type_of_service"])

In [ ]:
melt_funding[melt_funding["ntd_id"].isin([sample_ids[1]])].sort_values(by=["year","mode","type_of_service"])

## aggregation group by
- farebox melt
    - PMT, UPT, VRH totals for urban, per year
    - PMT, UPT, VRH totals for rural, per year
    - PMT, UPT, VRH totals for cohort A, per year
    - PMT, UPT, VRH totals for cohort B, per year
    - PMT, UPT, VRH totals for cohort C, per year
- funding melt
    - PMT, UPT, VRH totals for urban, per year
    - PMT, UPT, VRH totals for rural, per year
    - PMT, UPT, VRH totals for cohort A, per year
    - PMT, UPT, VRH totals for cohort B, per year
    - PMT, UPT, VRH totals for cohort C, per year

In [ ]:
group_list_agg = [
    "source_agency",
    "year",
    "ntd_id",
    "caltrans_district",
    
]
farebox_vrh_total = (
    melt_farebox[melt_farebox["ntd_metric"] == "total_vrh"]
    .groupby(group_list_agg)["ntd_metric_value"]
    .sum()
    .reset_index()
).rename(columns={"ntd_metric_value": "total_vrh"})

farebox_upt_total = (
    melt_farebox[melt_farebox["ntd_metric"] == "total_upt"]
    .groupby(group_list_agg)["ntd_metric_value"]
    .sum()
    .reset_index()
).rename(columns={"ntd_metric_value": "total_upt"})

farebox_pmt_total = (
    melt_farebox[melt_farebox["ntd_metric"] == "total_pmt"]
    .groupby(group_list_agg)["ntd_metric_value"]
    .sum()
    .reset_index()
).rename(columns={"ntd_metric_value": "total_pmt"})

farebox_cohort_totals = (
    cohort_merge_farebox.groupby(["year","cohort"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
)

farebox_area_totals = (
    cohort_merge_farebox.groupby(["year","urban_rural"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
)

In [ ]:
funding_vrh_total = (
    melt_funding[melt_funding["ntd_metric"] == "total_vrh"]
    .groupby(group_list_agg)["ntd_metric_value"]
    .sum()
    .reset_index()
).rename(columns={"ntd_metric_value": "total_vrh"})

funding_upt_total = (
    melt_funding[melt_funding["ntd_metric"] == "total_upt"]
    .groupby(group_list_agg)["ntd_metric_value"]
    .sum()
    .reset_index()
).rename(columns={"ntd_metric_value": "total_upt"})

funding_pmt_total = (
    melt_funding[melt_funding["ntd_metric"] == "total_pmt"]
    .groupby(group_list_agg)["ntd_metric_value"]
    .sum()
    .reset_index()
).rename(columns={"ntd_metric_value": "total_pmt"})

funding_cohort_totals = (
    cohort_merge_funding.groupby(["year","cohort"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
)

funding_area_totals = (
    cohort_merge_funding.groupby(["year","urban_rural"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
)

## summary stats

In [ ]:
melt_farebox.columns

In [ ]:
metric_df ={ 
    "farebox_upt_total":farebox_upt_total, 
    "farebox_pmt_total":farebox_pmt_total, 
    "farebox_vrh_total":farebox_vrh_total, 
    "funding_upt_total":funding_upt_total, 
    "funding_pmt_total":funding_pmt_total, 
    "funding_vrh_total":funding_vrh_total,

}
metric_cols = [
    "total_upt",
    "total_pmt",
    "total_vrh"
]
cohorts = [
    "Group A",
    "Group B",
    "Group C"
]


# Raw Data tables

In [ ]:
display(
    farebox_cohort_totals,
    funding_cohort_totals,
)

In [ ]:
display(
    farebox_area_totals,
    funding_area_totals
)

# Whats going on in 2022 for Group C?
- why do we see a jump in metrics data in 2022 for Group C?

In [ ]:
# who are the big contributors in group c?

In [ ]:
melt_farebox.plot()

In [ ]:
# how many unique NTD IDs in group c appear per year?

In [ ]:
melt_farebox.groupby(["cohort","year",]).agg({"ntd_id":"nunique"})

In [ ]:
melt_funding.groupby(["cohort","year",]).agg({"ntd_id":"nunique"})

In [ ]:
melt_farebox.groupby("year").agg({
    "ntd_id":"nunique",
})

In [ ]:
#
for metric in metric_cols:
    for cohort in cohorts:
        display(melt_farebox[
            (melt_farebox["ntd_metric"]==metric)
            & (melt_farebox["cohort"]==cohort)
            ].boxplot(column="ntd_metric_value"))

In [ ]:
melt_farebox[
            (melt_farebox["ntd_metric"]=="total_upt")
            & (melt_farebox["cohort"]=="Group C")
            ].boxplot(column="ntd_metric_value")

In [ ]:
melt_farebox.boxplot(column=["ntd_metric_value"], by="cohort")

In [ ]:
melt_farebox[melt_farebox["ntd_metric"]=="total_upt"]